In [7]:
import recordlinkage
from recordlinkage.datasets import load_febrl4, load_febrl2

In [12]:

dfA = load_febrl2()
dfB = load_febrl4()

In [11]:
dfA

given_name    surname street_number         address_1  \
rec_id                                                                
rec-2778-org       sarah      bruhn            44     forbes street   
rec-712-dup-0      jacob     lanyon             5        milne cove   
rec-1321-org     brinley  efthimiou            35  sturdee crescent   
rec-3004-org     aleisha     hobson            54     oliver street   
rec-1384-org       ethan    gazzola            49    sheaffe street   
...                  ...        ...           ...               ...   
rec-1487-org      thomas      green            44     tuthill place   
rec-1856-org       james    mcneill            42  archibald street   
rec-3307-org       paige       lock             7  a'beckett street   
rec-227-org      antonio    collier            25      govett place   
rec-1143-org       harry       ryan            30  van raalte place   

                address_2              suburb postcode state date_of_birth  \
rec_id                                                                       
rec-2778-org   wintersloe        kellerberrin     4510   vic      19300213   
rec-712-dup-0     wellwod  beaconsfield upper     2602   vic      19080712   
rec-1321-org    tremearne         scarborough     5211   qld      19940319   
rec-3004-org    inglewood           toowoomba     3175   qld      19290427   
rec-1384-org   bimby vale          port pirie     3088    sa      19631225   
...                   ...                 ...      ...   ...           ...   
rec-1487-org   holmeleigh         bonny hills     4740   vic      19420210   
rec-1856-org          NaN          evans head     2250   nsw      19011207   
rec-3307-org      camboon      carina heights     2290   nsw      19871002   
rec-227-org     the rocks         broken hill     2304   qld      19400225   
rec-1143-org        anana        preston west     2572    wa      19250425   

              soc_sec_id  
rec_id                    
rec-2778-org     7535316  
rec-712-dup-0    9497788  
rec-1321-org     6814956  
rec-3004-org     5967384  
rec-1384-org     3832742  
...                  ...  
rec-1487-org     9334580  
rec-1856-org     4837378  
rec-3307-org     5142242  
rec-227-org      3973395  
rec-1143-org     6392122  

[5000 rows x 10 columns]

In [15]:

# Indexation step
indexer = recordlinkage.Index()
indexer.block(left_on='given_name')
candidate_links = indexer.index(dfA)


In [31]:
# The Sorted Neighbourhood Index method is a great method when there is 
# relatively large amount of spelling mistakes.
indexer_sorted = recordlinkage.SortedNeighbourhoodIndex(left_on='given_name', right_on='given_name')
#indexer_sorted.block(left_on='given_name')
candidate_links = indexer_sorted.index(dfA, dfA)

In [32]:
print(len(candidate_links))
print(type(candidate_links))

147421
<class 'pandas.core.indexes.multi.MultiIndex'>


In [33]:
print(candidate_links)

MultiIndex([(  'rec-2778-org', 'rec-3721-dup-0'),
            (  'rec-1050-org', 'rec-3721-dup-0'),
            ('rec-2254-dup-2', 'rec-3721-dup-0'),
            (  'rec-1385-org', 'rec-3721-dup-0'),
            ( 'rec-370-dup-0', 'rec-3721-dup-0'),
            (  'rec-3344-org', 'rec-3721-dup-0'),
            (  'rec-1295-org', 'rec-3721-dup-0'),
            (   'rec-909-org', 'rec-3721-dup-0'),
            (  'rec-2672-org', 'rec-3721-dup-0'),
            (  'rec-2254-org', 'rec-3721-dup-0'),
            ...
            ('rec-3565-dup-1',   'rec-2909-org'),
            (  'rec-2840-org', 'rec-2733-dup-1'),
            ( 'rec-694-dup-1',   'rec-3127-org'),
            ( 'rec-694-dup-1',   'rec-2746-org'),
            (  'rec-2206-org',   'rec-3630-org'),
            (  'rec-2206-org',    'rec-763-org'),
            (  'rec-2206-org',    'rec-654-org'),
            (  'rec-2206-org',   'rec-3655-org'),
            (  'rec-2206-org',   'rec-2095-org'),
            (  'rec-2206-org',   '

In [34]:
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA)

In [35]:
# Classification step
matches = features[features.sum(axis=1) > 3]
print(len(matches))

7443


In [36]:
matches.head(10)


given_name  surname  date_of_birth  suburb  \
rec_id_1       rec_id_2                                                     
rec-3703-dup-1 rec-3703-org             0      1.0              1       1   
               rec-3703-dup-0           0      1.0              1       0   
               rec-3703-dup-3           0      0.0              1       1   
               rec-3703-dup-2           0      1.0              1       1   
rec-2436-org   rec-2436-dup-0           0      1.0              1       1   
rec-833-org    rec-833-dup-1            0      1.0              1       1   
rec-833-dup-0  rec-833-dup-1            0      1.0              1       0   
rec-151-org    rec-151-dup-0            0      1.0              1       0   
rec-651-org    rec-651-dup-2            0      1.0              1       1   
rec-651-dup-1  rec-651-dup-2            0      1.0              1       1   

                               state  address_1  
rec_id_1       rec_id_2                          
rec-3703-dup-1 rec-3703-org        1        1.0  
               rec-3703-dup-0      1        1.0  
               rec-3703-dup-3      1        1.0  
               rec-3703-dup-2      1        1.0  
rec-2436-org   rec-2436-dup-0      1        1.0  
rec-833-org    rec-833-dup-1       1        1.0  
rec-833-dup-0  rec-833-dup-1       1        1.0  
rec-151-org    rec-151-dup-0       1        1.0  
rec-651-org    rec-651-dup-2       1        1.0  
rec-651-dup-1  rec-651-dup-2       1        1.0